In [1]:
import struct as st
import datetime

In [2]:
# Load sample IRT data
f = open("../data/ZENITH_220411_230827.IRT", "rb")
bin = f.read()

In [3]:
# Define getter
def b_unpack(bd, pos, type='i'):
    if type in ['i', 'f']:
        return st.unpack(type, bd[pos:pos+4])
    elif type == 'd':
        return st.unpack(type, bd[pos:pos+8])
    elif type == 'b':
        return st.unpack(type, bd[pos:pos+1])
    else:
        raise TypeError("Invalid Type Reference")

In [4]:
# IRT file code (671112496)
print(b_unpack(bin, 0)[0])

671112496


In [5]:
# Number of recorded samples
sample_n = b_unpack(bin, 4)[0]
print(sample_n)

1255


In [6]:
# Number of recorded IRRs
rec_n = b_unpack(bin, 20)[0]
print(rec_n)

2


 The above two variables give the number of rows for the resulting data record, which in this case is header + sample_n * rec_n = 1 + 1255 * 2 = 2511.

In [7]:
# Get the two (= rec_n) frequencies from the record
for i in range(rec_n):
    print(b_unpack(bin, 24+(i*4), type='f'))

(11.100000381469727,)
(12.0,)


In [8]:
# Define start of the data dump
pos_0 = 24 + (4 * rec_n)

In [9]:
def get_date(time):
    d_0 = datetime.datetime(2001, 1, 1)
    return d_0 + datetime.timedelta(seconds=time)


In [10]:
# Get and translate time
time = b_unpack(bin, pos_0, type='i')[0]

print(get_date(time))

2022-04-11 23:08:27


In [11]:
# Get data row
rf = b_unpack(bin, pos_0+4, type='b')[0]
irt_0 = b_unpack(bin, pos_0+5, type='f')[0]
irt_1 = b_unpack(bin, pos_0+9, type='f')[0]
ang = b_unpack(bin, pos_0+13, type='f')[0] / 1.e3

if ang > 0:
    elev = (ang - int(ang)) * 1.e3
else:
    elev = (int(ang) - ang) * 1.e3